# MISA
Alohan'ny mamerina dia avereno atao Run ny notebook iray manontolo. Ny fanaovana azy dia redémarrena mihitsy ny kernel aloha (jereo menubar, safidio **Kernel$\rightarrow$Restart Kernel and Run All Cells**).

Izay misy hoe `YOUR CODE HERE` na `YOUR ANSWER HERE` ihany no fenoina. Afaka manampy cells vaovao raha ilaina. Aza adino ny mameno references eo ambany raha ilaina.

## References
Eto ilay references rehetra no apetraka

---

In [2]:
import cvxpy as cp
import pandas as pd
import numpy as np
from sklearn.datasets import load_diabetes
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

def rel_error(x, y):
    """ returns relative error """
    return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

data = load_diabetes()
X_train, y_train = data.data, data.target

# Kernel Ridge Regression

In [3]:
def gaussian_kernel(x1, x2, sigma=1.0):
    return np.exp(- np.sum((x1 - x2)**2) / sigma)

class KernelRidgeRegression():
    def __init__(self, kernel, lambd=1.0):
        self.lambd = lambd # regularization coefficient
        self.alpha = None # dual variable
        self.kernel = kernel # kernel function
    
    def fit(self, X, y):
        
        N, d = X.shape
        
        K = np.zeros((N,N))
        
        for i in range(N):
            for j in range(N):
                K[i, j] = self.kernel(X[i], X[j])
        
        A = K + self.lambd * np.eye(N)
        Q, R = np.linalg.qr(A)
        
        self.alpha = np.linalg.solve(R, Q.T @ y)
        self.w = X.T @ self.alpha
        
    def predict(self, X):
        return X @ self.w

In [4]:
kr = KernelRidgeRegression(kernel=gaussian_kernel)
kr.fit(X_train,y_train)
y_pred = kr.predict(X_train)

sklearn_kr = KernelRidge(kernel=gaussian_kernel)
sklearn_kr.fit(X_train,y_train)
sklearn_y_pred = kr.predict(X_train)

error = rel_error(kr.alpha, sklearn_kr.dual_coef_)
print("error: ", error)
assert error <= 1e-11

error_pred = rel_error(y_pred, sklearn_y_pred)
print("prediction error: ", error_pred)
assert error_pred <= 1e-11

error:  5.692326333690524e-13
prediction error:  0.0


# Non-negative Linear Regression

In [27]:
class NonNegativeLinearRegression:
    def __init__(self, fit_intercept=True):
        self.w = None
        self.fit_intercept = fit_intercept
    
    def fit(self, X, y):
        
        X =  np.hstack( (np.ones((X.shape[0],1)), X) ) if self.fit_intercept else X.copy()
        
        N, d = X.shape
        
        w = cp.Variable(d, nonneg=True)
        
        objective = cp.Minimize(cp.sum_squares(X@w - y))
       
        problem = cp.Problem(objective)
        problem.solve(verbose=True)
    
        self.w = w.value

    def predict(self, X):
        X =  np.hstack( (np.ones((X.shape[0],1)), X) ) if self.fit_intercept else X.copy()
        return X @ self.w

In [31]:
nnlr = NonNegativeLinearRegression(fit_intercept=True)
nnlr.fit(X_train,y_train)
y_pred = kr.predict(X_train)
mse = mean_squared_error(y_pred, y_train)

sk_model = LinearRegression(fit_intercept=True)
sk_model.fit(X_train, y_train)
sk_pred = sk_model.predict(X_train)
sk_mse = mean_squared_error(sk_pred, y_train)

print("MSE scikit-learn:", sk_mse)
print("MSE gradient descent model :", mse)
print(f"Difference {abs(sk_mse - mse)}")

                                     CVXPY                                     
                                     v1.5.3                                    
(CVXPY) Oct 26 09:57:33 AM: Your problem has 11 variables, 0 constraints, and 0 parameters.
(CVXPY) Oct 26 09:57:33 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Oct 26 09:57:33 AM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Oct 26 09:57:33 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Oct 26 09:57:33 AM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Oct 26 09:57:33 AM: Compiling problem (target solver=OSQP).
(CVX

In [28]:
nnlr = NonNegativeLinearRegression(fit_intercept=True)
nnlr.fit(X_train,y_train)
y_pred = kr.predict(X_train)
mse = mean_squared_error(y_pred, y_train)

sk_model = LinearRegression(fit_intercept=True, positive=True)
sk_model.fit(X_train, y_train)
sk_pred = sk_model.predict(X_train)
sk_mse = mean_squared_error(sk_pred, y_train)

print("MSE scikit-learn:", sk_mse)
print("MSE gradient descent model :", mse)
assert mse - sk_mse < 200

                                     CVXPY                                     
                                     v1.5.3                                    
(CVXPY) Oct 26 09:54:26 AM: Your problem has 11 variables, 0 constraints, and 0 parameters.
(CVXPY) Oct 26 09:54:26 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Oct 26 09:54:26 AM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Oct 26 09:54:26 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Oct 26 09:54:26 AM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Oct 26 09:54:26 AM: Compiling problem (target solver=OSQP).
(CVX

AssertionError: 